In [3]:
# Import required libraries and set up our environment
# We'll use these throughout the tutorial to interact with AWS S3
import os
import pprint
import pandas as pd
import boto3

print("📚 Setting up the environment...")

# Initialize pretty printer for better output formatting
pp = pprint.PrettyPrinter(indent=2)

# Create S3 client using default credentials from AWS CLI
# boto3 will automatically use credentials from ~/.aws/credentials
s3 = boto3.client(
    "s3",
    region_name="eu-west-1",  # Ireland region
)

print("✅ Environment setup complete!")
print(f"🌍 Using AWS region: {s3.meta.region_name}")

def fetch_latest_data(bucket_name, folder_prefix):
    # 2. List objects in that folder to find the actual filename
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

    # 3. Find the latest file (ignoring the folder placeholder itself)
    # We sort by 'LastModified' to ensure we get the newest export
    files = response.get('Contents', [])
    # Filter out the folder itself and get files
    data_files = [f for f in files if f['Key'] != folder_prefix]

    if not data_files:
        print("❌ No files found. Did the Athena query finish?")
    else:
        # Get the most recent file
        latest_file = max(data_files, key=lambda x: x['LastModified'])
        file_key = latest_file['Key']
        
        print(f"✅ Found file: {file_key}")
        
        # 4. Download and Load into Python
        # We download to a temporary file first
        local_filename = "downloaded_data.json.gz"
        s3.download_file(bucket_name, file_key, local_filename)
        
        print("⬇️ Download complete. Loading into DataFrame...")
        
        # 5. Read into Pandas 
        return pd.read_json(local_filename)
    
monthly = fetch_latest_data("business-news-sentiments", "news_sentiments_monthly/combined_networks/")

📚 Setting up the environment...
✅ Environment setup complete!
🌍 Using AWS region: eu-west-1
✅ Found file: news_sentiments_monthly/combined_networks/20251215_201146_00304_2w73e_a46e0eb0-7256-4e7c-9ab8-cd475b644304
⬇️ Download complete. Loading into DataFrame...


BadGzipFile: Not a gzipped file (b'{"')